In [1]:
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.layers import Activation, Input, Embedding, LSTM, Dense, Lambda, GaussianNoise, concatenate
from keras.models import Model
import numpy as np
from keras.utils import np_utils, to_categorical
from keras.layers.core import Dense, Dropout, Activation
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Add
from keras.optimizers import SGD, Adam, RMSprop
from keras.constraints import max_norm
from keras.layers import MaxPooling2D, Dropout, Dense, Flatten, Activation, Conv2D
from keras.models import Sequential
from keras.losses import categorical_crossentropy as logloss
from keras.metrics import categorical_accuracy
from keras.preprocessing import image
import matplotlib.pyplot as plt
import glob
from PIL import Image
import imageio
import cv2
import pandas as pd
import collections
import os
import natsort
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import concurrent.futures
import pickle
import os.path

Using TensorFlow backend.


In [55]:
# nb_classes = 10

# (X_train, y_train), (X_test, y_test) = mnist.load_data()

# plt.imshow(X_train[3])
# print(y_train[3])

# # convert y_train and y_test to categorical binary values 
# Y_train = np_utils.to_categorical(y_train, nb_classes)
# Y_test = np_utils.to_categorical(y_test, nb_classes)

# print(Y_train[3])
# X_train = X_train.reshape(60000, 28, 28, 1)
# X_test = X_test.reshape(10000, 28, 28, 1)

# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')

# # Normalize the values
# X_train /= 255
# X_test /= 255

#########################################################


# def processing_img(filename):
#     img = image.load_img('./myntradataset/images/'+filename+'.jpg', target_size=(60,80,3))
#     img = image.img_to_array(img)
#     img = img/255
#     return img

# list_id = list(styles_data['id'].astype('str'))
    
# with concurrent.futures.ProcessPoolExecutor() as executor:
#     for file_id , img in tqdm(zip(list_id ,executor.map(processing_img, list_id )), total=len(list_id)):
#         matrix_images.append(img)



#########################################################


# filenames = glob.glob("./myntradataset/images/*.jpg")

# filenames = natsort.natsorted(filenames)

# data = []
# length_data = 1000

# for i in range(length_data):
#     image = cv2.imread(filenames[i])
#     widht = 60
#     height = 80
#     dim = (widht,height)
#     image = cv2.resize(image, dim) 
#     data.append(image)

# data = np.array(data) 
# data = data.astype('float32')
# data = data / 255
# # print(data.shape)

# styles_data = pd.read_csv('myntradataset/styles.csv', error_bad_lines=False)

# styles_data.sort_values(by=['id'], inplace=True)
# styles_data = styles_data.iloc[:,2:3]
# styles_data = styles_data.to_numpy()
# styles_data = styles_data.flatten()

# labels = []
# mydict = collections.defaultdict(lambda : -1) 
# inc_tag = 0

# for i in range(length_data):
#     if mydict[styles_data[i]] == -1:
#         mydict[styles_data[i]] = inc_tag
#         inc_tag += 1
#         labels.append(mydict[styles_data[i]])
#     else:
#         labels.append(mydict[styles_data[i]])

# # print(mydict)
# nb_classes = 4
# labels = np_utils.to_categorical(labels, nb_classes)        
# # print(labels.shape)

# input_shape = (80,60,3)

# X_train, X_test,y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)
# # print(X_test.shape)


# import numpy as np
# testArray = np.random.rand(100, 9)

# for n in range(9):
#     print(n)
#     testArray = np.delete(testArray,[n],axis=1)
#     print(testArray.shape)
#     print(testArray[0:1][:])

# testArray = np.delete(testArray,[1,2,3])
# print(testArray.shape)
# print(testArray[0:1][:])

# pre-processing dataset

In [2]:
y_data = None
x_data = None
nb_classes = None

if os.path.isfile("X.pickle") and os.path.isfile("Y.pickle"):
    
    file_pickle = open("X.pickle", "rb")
    x_data = pickle.load(file_pickle)
    file_pickle.close()
    
    file_pickle = open("Y.pickle", "rb")
    y_data = pickle.load(file_pickle)
    file_pickle.close()
    
    nb_classes = y_data.shape[1]

else:
    
    styles_data = pd.read_csv('myntradataset/styles.csv', error_bad_lines=False)
    matrix_images = []

#     styles_data = styles_data.iloc[:600]
    styles_data.articleType, uniques_class = pd.factorize(styles_data.articleType)
    uniques_class = np.array(uniques_class)
    nb_classes = len(uniques_class)
    y_data = styles_data['articleType'].values
    y_data = to_categorical(y_data)
    
    missing_file = ["39403",
                    "39410",
                    "39401",
                    "39425"]
    
    drop_index = []

    for i in tqdm(range(styles_data.shape[0])):
#     for i in tqdm(range(600)):
        filename = styles_data['id'][i].astype('str')
        if filename not in missing_file:
            img = image.load_img('./myntradataset/images/'+styles_data['id'][i].astype('str')+'.jpg', target_size=(60,80,3))
            img = image.img_to_array(img)
            img = img/255
            matrix_images.append(img)
        else:
            drop_index.append(i)
            
    x_data = np.array(matrix_images)
    y_data = np.delete(y_data, drop_index, axis=0)

    file_pickle = open("X.pickle","wb")
    pickle.dump(x_data, file_pickle)
    file_pickle.close()
    
    file_pickle = open("Y.pickle","wb")
    pickle.dump(y_data, file_pickle)
    file_pickle.close()

print(x_data.shape, y_data.shape, nb_classes)
        

(44420, 60, 80, 3) (44420, 143) 143


In [3]:
X_train, X_test,y_train, y_test = train_test_split(x_data, y_data, test_size=0.33, random_state=42)
input_shape = (60,80,3)

#  build teacher model

In [4]:
def build_teacher_model():
#     filters = 64 or 128 
    pool_size = (2,2)
    kernel_size = (3,3)
    
    teacher_m = Sequential()
    teacher_m.add(Conv2D(
        32,
        kernel_size = kernel_size,
        activation='relu',
        input_shape=input_shape
    ))
    
    teacher_m.add(Conv2D(64,kernel_size=kernel_size, activation ='relu'))
    
    teacher_m.add(MaxPooling2D(pool_size = pool_size))
    teacher_m.add(Dropout(0.25))
    teacher_m.add(Flatten())
    teacher_m.add(Dense(128, activation = 'relu'))
    teacher_m.add(Dropout(0.5))
    teacher_m.add(Dense(nb_classes))
    teacher_m.add(Activation('softmax'))
    
    teacher_m.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
    
    return teacher_m

teacher_m = build_teacher_model()
print(teacher_m.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 58, 78, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 76, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 38, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 38, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 68096)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8716416   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

# build student model

In [5]:
def build_student_model():
    
    student = Sequential()
    student.add(Flatten(input_shape=input_shape))
    student.add(Dense(32, activation='relu'))
    student.add(Dropout(0.2))
    student.add(Dense(nb_classes))
    student.add(Activation('softmax'))

    #sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    student.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    return student

student = build_student_model()
student.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 14400)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                460832    
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 143)               4719      
_________________________________________________________________
activation_2 (Activation)    (None, 143)               0         
Total params: 465,551
Trainable params: 465,551
Non-trainable params: 0
_________________________________________________________________


In [69]:
# Train the teacher model as usual
epochs = 10
batch_size = 56
teacher_m.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1, validation_data = (X_test, y_test))

Train on 29761 samples, validate on 14659 samples
Epoch 1/10
29761/29761 [==============================] - 260s 9ms/step - loss: 2.6967 - accuracy: 0.3846 - val_loss: 1.3548 - val_accuracy: 0.6585
Epoch 2/10
29761/29761 [==============================] - 263s 9ms/step - loss: 1.5117 - accuracy: 0.6144 - val_loss: 1.0441 - val_accuracy: 0.7229
Epoch 3/10
29761/29761 [==============================] - 263s 9ms/step - loss: 1.2250 - accuracy: 0.6762 - val_loss: 0.9283 - val_accuracy: 0.7455
Epoch 4/10
29761/29761 [==============================] - 263s 9ms/step - loss: 1.0574 - accuracy: 0.7090 - val_loss: 0.8752 - val_accuracy: 0.7601
Epoch 5/10
29761/29761 [==============================] - 264s 9ms/step - loss: 0.9513 - accuracy: 0.7351 - val_loss: 0.7937 - val_accuracy: 0.7752
Epoch 6/10
29761/29761 [==============================] - 264s 9ms/step - loss: 0.8522 - accuracy: 0.7619 - val_loss: 0.7761 - val_accuracy: 0.7857
Epoch 7/10
29761/29761 [==============================] - 262s

##### 